# DataDump Prep 

This is used to create the mapping for ward code, ward population and ward level IMD
 
For each nation in the UK, the relevant statistics authority has created a ranking of multiple deprivation that ranks small census areas (LSOAs in England/Wales, DZs in Scotland and SOA in Northern Ireland). There is no equivalent ranking of areas across the entire UK.

These indexes are independent and cannot be directly compared. We are using mysociety's python package mysoc_dataset. For more inforamtion, plase see https://pages.mysociety.org/composite_uk_imd/datasets/uk_index/latest 

We have chosen “UK_IMD_E.csv” as the majority of data is English. When in a dataset for a particular nation (for instance, uk_index/UK_IMD_E.csv), the rank order of the original deprivation index is preserved, but it is not for other countries.


In [1]:
import pandas as pd
import numpy as np
from mysoc_dataset import get_dataset_df

## Map Ward Boundaries to LSOA

We are going to use the ward boundaries to create the map on our dashboard. Therefore, it is important to make sure that we map the lsoa to the same list as the ward boundaries (https://geoportal.statistics.gov.uk/search?q=BDY_WD%202023&sort=Title%7Ctitle%7Casc).

https://geoportal.statistics.gov.uk/datasets/e14b1475ecf74b58804cf667b6740706/about 
This is used for the ward to LSOA Mapping. Ward and LSOA do not have a 1:1 mapping and the boundaries can change over time. We are better off using ONS mapping than creating our own. 

When you download the full data collection, there’s a lot of files. What we are after is the extended postcode list under “data”. There are other files with IMD data and other mapping but they are not as extensive (e.g. some of them didn’t have mapping for the highlands).
The “ONSPD_FEB_2024_UK.csv” contains the most extensive mapping at all levels. This is a very large file so you are better off using a script to extract the data frame required (see the code below)



In [2]:
# Load only the lsoa nad ward columns from the ONSPD dataset and drop duplicates
lsoa_map_columns = ['osward', 'lsoa11']  # Specify the columns you need
lsoa_map_df= pd.read_csv('data/ONSPD_FEB_2024_UK.csv', usecols=lsoa_map_columns)
lsoa_map_df = lsoa_map_df.drop_duplicates()



In [3]:
# Use mysoc_dataset to load the composite_uk_imd dataset
imd_df = get_dataset_df(
    repo_name="composite_uk_imd",
    package_name="uk_index",
    version_name="latest",
    file_name="UK_IMD_E.csv",
    )

imd_with_ward_df = imd_df.merge(lsoa_map_df, left_on='lsoa', right_on='lsoa11', how='left')

# Clean the data and get the lowest IMD values for each ward
lowest_values_imd_with_ward_df = imd_with_ward_df.groupby('osward').agg({
    'UK_IMD_E_score': 'min',
    'original_decile': 'min',
    'E_expanded_decile': 'min',
    'UK_IMD_E_rank': 'min',
    'UK_IMD_E_pop_decile': 'min',
    'UK_IMD_E_pop_quintile': 'min'
})



If you find uk_index helpful, can you tell us how using this survey? 
https://pages.mysociety.org/composite_uk_imd/downloads/uk-index-constituency-imd-csv/latest#survey. This message 
can be removed by setting the `done_survey` option to True.

In [4]:
# Use the ward boundaries dataset and left join with the lsoa_map_df to get the lsoa for each ward
ward_boundaries_df = pd.read_csv('data/Wards_December_2023_Boundaries_UK_BGC_-4058071213750345486.csv')
ward_boundaries_imd_df = ward_boundaries_df.merge(lowest_values_imd_with_ward_df, left_on='WD23CD', right_on='osward', how='left')


## Get Population data

The ward level population data are not stored in one place. 

Northern Ireland Ward Level Population Data (2021) - https://www.nisra.gov.uk/publications/census-2021-main-statistics-settlements-and-wards-northern-ireland

Scotland Ward Level Population Data (2021) - https://www.nrscotland.gov.uk/statistics-and-data/statistics/statistics-by-theme/population/population-estimates/2011-based-special-area-population-estimates/electoral-ward-population-estimates

England and Wales Ward Level Population Data (2022) - https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/wardlevelmidyearpopulationestimatesexperimental



In [8]:
# Load the NI population data
ni_pop_df = pd.read_excel('data/census-2021-ms-a01.xlsx', sheet_name='Ward', header=5)
#load the Scotland population data
scot_pop_df = pd.read_excel('data/electoral-wards-21-tabs.xlsx', sheet_name='2021', header=3)
scot_pop_df = scot_pop_df.loc[scot_pop_df['Sex'] == 'Persons', scot_pop_df.columns[:4]]
#load the England and Wales population data
e_and_w_pop_df_columns = ['Ward 2023 Code', 'Ward 2023 Name', 'Total']
e_and_w_pop_df = pd.read_excel('data/sapewardstablefinal.xlsx', sheet_name='Mid-2022 Ward 2023', header=3, usecols=e_and_w_pop_df_columns)
e_and_w_pop_df

ni_pop_df = ni_pop_df[['Geography code', 'Geography', 'All usual residents']].rename(columns={'Geography code': 'code', 'Geography': 'name', 'All usual residents': 'total_Population'})
scot_pop_df = scot_pop_df[['Electoral Ward 2022 Code', 'Electoral Ward 2022 Name', 'Total']].rename(columns={'Electoral Ward 2022 Code': 'code', 'Electoral Ward 2022 Name': 'name', 'Total': 'total_Population'})
e_and_w_pop_df = e_and_w_pop_df[['Ward 2023 Code', 'Ward 2023 Name', 'Total']].rename(columns={'Ward 2023 Code': 'code', 'Ward 2023 Name': 'name', 'Total': 'total_Population'})

#create 1 dataframe with all the population data
uk_pop_df = pd.concat([ni_pop_df, scot_pop_df, e_and_w_pop_df])

## Create Datadump csv with ward level IMD and Population

In [10]:
result_df = ward_boundaries_imd_df.merge(uk_pop_df, left_on='WD23CD', right_on='code', how='left')
result_df = result_df[['WD23CD', 'WD23NM', 'WD23NMW', 'UK_IMD_E_score', 'original_decile', 'E_expanded_decile', 'UK_IMD_E_rank', 'UK_IMD_E_pop_decile', 'UK_IMD_E_pop_quintile', 'total_Population']]
result_df.to_csv('data/datadump_ward_imd.csv', index=False)